Script to output the U100 data for the VWIND constraint for CMIP5.  This includes ensemble mean 100hPa zonal mean zonal wind averaged from 20N to 40N for the future-past difference for the ensemble mean and for 1 mem.

In [11]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import xesmf as xe
from numpy import nan
import sys
import warnings

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import averaging_utils as avg

importlib.reload(read)
importlib.reload(cal)
importlib.reload(avg)

warnings.filterwarnings('ignore')



Set paths for CMIP5 models (historical and RCP8.5) and the pressure level to us (in Pa)

In [12]:
histpath="/project/cmip5/historical/Amon/"
rcp85path="/project/cmip5/rcp85/Amon/"
plevuse="10000"
pathout="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

Information ont he models being used is provided in cmip5csvinfo.csv.  This contains information on the models, number of members and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [13]:
cmip5models=pd.read_csv("../cmip5csvinfo.csv")

ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for past period
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for Future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

In [14]:
models=cmip5models["Model"]
nmodels=models.size

# define 1 deg greid
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

# define multi-model arrays
upast_em = xr.DataArray(np.zeros([nmodels,nlat]), coords=[models, grid_out['lat']], dims=['Model', 'lat'], name='upast_em')
upast_1mem = xr.DataArray(np.zeros([nmodels,nlat]), coords=[models, grid_out['lat']], dims=['Model', 'lat'], name='upast_1mem')
ufut_em = xr.DataArray(np.zeros([nmodels,nlat]), coords=[models, grid_out['lat']], dims=['Model', 'lat'], name='ufut_em')
ufut_1mem = xr.DataArray(np.zeros([nmodels,nlat]), coords=[models, grid_out['lat']], dims=['Model', 'lat'], name='ufut_1mem')
# loop over models
for index, modname in models.iteritems():
    
    # initialize boolean for reusing weights
    
    # ---- the past period ----
    nmems=cmip5models.loc[index, "Nmempast"]
    
    for imem in range(1, nmems+1, 1): # loop over members
        memstr="r"+str(imem)+"i1p1"
        
        # check for a special order
        changeorder = cmip5models.loc[index, "specialorderpast"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing past for "+modname+" "+memstr+"...")
        histdir=histpath+"ua/"+modname+"/"+memstr+"/"
        rcp85dir=rcp85path+"ua/"+modname+"/"+memstr+"/"
        
        # read in meridional wind, concatenate historical to 2005, rcp85 to 2014
        uhist = read.read_zonalmean_1lev(histdir+"/*.nc", datebegp, "2005-12", plevuse)
        urcp = read.read_zonalmean_1lev(rcp85dir+"/*.nc", "2006-01", dateendp, plevuse)
        u=xr.concat([uhist, urcp], dim="time", join="override") 

        # check the array length.  If this fiales, try ending historical in 2005-11
        # and beginning rcp in 2005-12 to fix Hadley center
        if (u.time.size != nmonthsp):
            print("switching to the assumption that historical ends at 2005-11")
            uhist = read.read_zonalmean_1lev(histdir+"/*.nc", datebegp, "2005-11", plevuse)
            urcp = read.read_zonalmean_1lev(rcp85dir+"/*.nc", "2005-12", dateendp, plevuse)
            u=xr.concat([uhist, urcp], dim="time", join="override")
            
        # check again
        if (u.time.size != nmonthsp):
            print("something's wrong, nmonthsp="+str(nmonthsp)+" but u has size "+str(v.time.size))
            sys.exit("u past for "+modname+", "+memstr+" doesn't have the right size")
            
        # calculate DJF mean and interpolate
        udjf = cal.season_mean(u, "ua", season="DJF")
        udjf = udjf.dropna('lat')
        udjfinterp = udjf.interp(lat=np.linspace(-90,90,181), method="cubic")
        
        if (imem == 1):
            upast_1mem[index,:] = udjfinterp
            
        upast_em[index,:] = upast_em[index,:] + udjfinterp[:]/float(nmems)
        
    # ---the future period-------
    
    nmems=cmip5models.loc[index, "Nmemfuture"]
    for imem in range(1,nmems+1,1):
        memstr="r"+str(imem)+"i1p1"
        changeorder=cmip5models.loc[index,"specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing future for "+modname+" "+memstr)
        rcp85dir=rcp85path+"/ua/"+modname+"/"+memstr+"/"
        u=read.read_zonalmean_1lev(rcp85dir+"/*.nc", datebegf, dateendf, plevuse)
        
        # check the right number of months are there
        if (u.time.size != nmonthsf):
            if (modname=="HadGEM2-ES"):
                u=u.isel(time=slice(0, u.time.size-1)) # taking care of another Hadley center problem
        if (u.time.size != nmonthsf):
            print("something's wrong, nmonthf="+str(nmonthsf)+" but u has size "+str(u.time.size))
            sys.exit("u future for "+modname+", "+memstr+" doesn't have the right size")
            
        udjf = cal.season_mean(u, "ua", season="DJF")
        udjf = udjf.dropna('lat')
        udjfinterp = udjf.interp(lat=np.linspace(-90,90,181), method="cubic")
        
        if (imem == 1):
            ufut_1mem[index,:]=udjfinterp
            
        ufut_em[index,:] = ufut_em[index,:] + udjfinterp[:]/float(nmems)

Processing past for ACCESS1-0 r1i1p1...
Processing future for ACCESS1-0 r1i1p1
Processing past for ACCESS1-3 r1i1p1...
Processing future for ACCESS1-3 r1i1p1
Processing past for bcc-csm1-1 r1i1p1...
Processing future for bcc-csm1-1 r1i1p1
Processing past for bcc-csm1-1-m r1i1p1...
Processing future for bcc-csm1-1-m r1i1p1
Processing past for BNU-ESM r1i1p1...
Processing future for BNU-ESM r1i1p1
Processing past for CanESM2 r1i1p1...
Processing past for CanESM2 r2i1p1...
Processing past for CanESM2 r3i1p1...
Processing past for CanESM2 r4i1p1...
Processing past for CanESM2 r5i1p1...
Processing future for CanESM2 r1i1p1
Processing future for CanESM2 r2i1p1
Processing future for CanESM2 r3i1p1
Processing future for CanESM2 r4i1p1
Processing future for CanESM2 r5i1p1
Processing past for CCSM4 r1i1p1...
Processing past for CCSM4 r2i1p1...
Processing past for CCSM4 r3i1p1...
Processing past for CCSM4 r4i1p1...
Processing past for CCSM4 r5i1p1...
Processing past for CCSM4 r6i1p1...
Processing

Calculate u100 index and output to netcdf

In [15]:
# V_SW index
u100_past_em = avg.cosweightlat(upast_em,20,40)
u100_fut_em = avg.cosweightlat(ufut_em,20,40)
u100 = u100_fut_em - u100_past_em
u100.to_netcdf(pathout+"u100_CMIP5.nc")